## 05. 의사결정나무란?

### 의사결정나무

* 목표 : 예측 변수를 기반으로 결과를 분류하거나 예측
* 결정 규칙(decision rule)을 나무구조(tree)로 도표화하여 분류(classification)와 예측(prediction)을 수행하는 분석방법

#### 주요 방법

* Trees and Rule 구조
    * 규칙(rules)은 나무 모델(tree diagram)로 표현
    * 결과는 규칙(rules)으로 표현
* 재귀적 분할(Recursive partitioning)
    * 나무를 만드는 과정
    * 그룹이 최대한 동질하도록 반복적으로 레코드를 하위 그룹으로 분리
* 가지치기(Pruning the tree)
    * 생성된 나무를 자르는 과정(정교화)
    * 과적합을 피하기 위해 필요 없는 가지를 간단히 정리  

#### 의사결정 나무 구분
* 구분
    * 분류나무 : 목표 변수가 범주형 변수
    * 회귀나무 : 목표 변수가 수치형 변수 (예측 모델이 더욱 잘 되어 있어 거의 사용 X)
* 재귀적 분할 알고리즘
    * CART(Classification And Regression Tree)
    * C4.5
    * CHAID(Chi-square Automatic Interaction Detection)
* 불순도(Impurity) 알고리즘
    * Gini index
    * Entropy index, 정보 이익(Informaion Gain)
    * 카이제곱 통계량(Chi-Square statistic)

#### 분류 나무(Classification Tree)

* 목표 변수 : 범주형 변수 (분리)  
* 예측 변수 : 범주형, 수치형 가능

|분류 알고리즘|불순수도 지표|
|---|---|
|CART|Gini index|
|C4.5|엔트로피, 정보이익, 정보이익비율|
|CHAID|카이제곱 통계량|

* 경향을 알아볼 때에도 사용 가능함

![의사결정나무 정리](https://media.vlpt.us/images/noooooh_042/post/5209bd2f-65b0-4a35-b3ac-1ecd0d5d248c/image-20200925011308304.png)

#### 의사결정나무 과정

1. 나무 모델 생성
2. 과적합 문제 해결
3. 검증
4. 해석 및 예측

#### 과적합 문제

* 과적합
    * 학습용 데이터에 완전히 적합
    * 학습용 집합에서 잡음(noise)도 모형화하기 때문에 평가용 집합에서 전체 오차는 일반적으로 증가하게 됨

* 과소적합(높은 편향)
    * 훈련, 검증 정확도가 모두 낮을 경우
    * **학습 곡선**을 이용
        * 학습곡선은 샘플 데이터의 수에 따른 정확도 변화를 의미
* 과대적합(높은 분산)
    * 훈련 데이터에 비해 모델이 너무 복잡할 경우
    * **검증 곡선**을 이용
        * 검증 곡선은 하이퍼 파라미터에 따른 정확도 변화를 의미

#### 과적합 방지

* 성장 멈추기 (사용X)
* 가지치기(Pruning the Tree)
    * 나무 모델 생성 후 필요없는 가지 제거
    * 성장 멈추기보다 더욱 뛰어난 성능
    * C4.5 : 학습 데이터를 이용하여 나무 모델 성장과 가지치기에 사용
    * CART : 학습 데이터는 나무 모델 성장에, 검증 데이터는 가지치기에 사용

#### 최적화

* 교차검정(Cross Validation)
    * Hold-out 교차검정
    * K-fold 교차검정
* Pipe line
    * 학습곡선과 검증곡선에서 정확도 변화 추적
* 하이퍼파라미터 튜닝
    * 그리드 서치를 사용한 머신 러닝 모델 세부 튜닝
    * 기계학습 모델의 성능을 결정하는 하이퍼 파라미터 튜닝

#### 의사결정나무 modeling

실제 modeling을 해보자.

In [1]:
# 기본
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # Graph

# 데이터 가져오기
import pandas as pd

# 데이터 전처리
from sklearn.preprocessing import StandardScaler    # 연속 변수 표준화
from sklearn.preprocessing import LabelEncoder      # 범주형 변수 수치화

# 훈련/검증용 데이터 분리
from sklearn.model_selection import train_test_split    # 훈련과 테스트를 위한 데이터 분리

# 분류 모델
from sklearn.tree import DecisionTreeClassifier     # 의사결정나무

# 모델 검정
from sklearn.metrics import confusion_matrix, classification_report # 정오분류표
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer  # 정확도, 민감도 등
from sklearn.metrics import roc_curve   # ROC 곡선

# 최적화
from sklearn.model_selection import cross_validate  # 교차 타당도
from sklearn.pipeline import make_pipeline  # 파이프라인 구축
from sklearn.model_selection import learning_curve, validation_curve # 학습곡선, 검증곡선
from sklearn.model_selection import GridSearchCV    # 하이퍼파라미터 튜닝

In [2]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

# train data의 상위 5개 출력
train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,73,0


In [3]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  awards_won?           54808 non-null  int64  
 11  avg_training_score    54808 non-null  int64  
 12  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 5.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23490 ent

In [4]:
# 범주형 변수 통계량 확인
train.describe(include = 'object')

,department,region,education,gender,recruitment_channel
count,54808,54808,52399,54808,54808
unique,9,34,3,2,3
top,Sales & Marketing,region_2,Bachelor's,m,other
freq,16840,12343,36669,38496,30446


In [5]:
# 결측치 확인

train_total = train.isnull().sum()
test_total = test.isnull().sum()

train_total , test_total

(employee_id                0
 department                 0
 region                     0
 education               2409
 gender                     0
 recruitment_channel        0
 no_of_trainings            0
 age                        0
 previous_year_rating    4124
 length_of_service          0
 awards_won?                0
 avg_training_score         0
 is_promoted                0
 dtype: int64,
 employee_id                0
 department                 0
 region                     0
 education               1034
 gender                     0
 recruitment_channel        0
 no_of_trainings            0
 age                        0
 previous_year_rating    1812
 length_of_service          0
 awards_won?                0
 avg_training_score         0
 dtype: int64)

In [6]:
# 결측치 변경

# train set
train['education'] = train['education'].fillna(train['education'].mode()[0])
train['previous_year_rating'] = train['previous_year_rating'].fillna(train['previous_year_rating'].mode()[0])

print("Number of Missing Values Left in the Training Data :", train.isnull().sum().sum())

# test set
test['education'] = test['education'].fillna(test['education'].mode()[0])
test['previous_year_rating'] = test['previous_year_rating'].fillna(test['previous_year_rating'].mode()[0])

print("Number of Missing Values Left in the Testing Data :", test.isnull().sum().sum())

Number of Missing Values Left in the Training Data : 0
Number of Missing Values Left in the Testing Data : 0


In [7]:
train.keys()

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'awards_won?', 'avg_training_score',
       'is_promoted'],
      dtype='object')

In [8]:
X = train.drop(['is_promoted'], axis=1)
X.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won?,avg_training_score
0,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,0,49
1,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,60
2,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,50
3,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,50
4,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,73


In [9]:
y = train['is_promoted']

np.bincount(y)

array([50140,  4668])

In [10]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                     test_size = 0.3,   # test set의 비율
                     random_state = 1,  # 무작위 시드 번호
                     stratify = y)      # 결과 레이블의 비율대로 분리

In [ ]:
# 의사결정나무 model 구축
tree = DecisionTreeClassifier(criterion='entropy',
                              max_depth=None,
                              random_state=1)
tree.fit(X_train, y_train)

In [ ]:
y_pred = tree.predict(X_test)

In [ ]:
confmat = pd.DataFrame(confusion_matrix(y_test, y_pred),
                       index=['True[0]', 'True[1]'],
                       columns = ['Predict[0]', 'Predict[1]'])

In [ ]:
print('Classification Report')
print(classification_report(y_test, y_pred))

In [ ]:
print(f'잘못 분류된 sample 갯수 : {(y_test != y_pred).sum()}')
print(f'정확도 : {accuracy_score(y_test, y_pred):.2f}%')
print(f'정밀도 : {precision_score(y_true=y_test, y_pred=y_pred):.3f}%')
print(f'재현율 : {recall_score(y_true=y_test, y_pred=y_pred):.3f}%')
print(f'F1 : {f1_score(y_true=y_test, y_pred=y_pred):.3f}%')

In [ ]:
import graphviz
import pydotplus
from pydotplus import graph_from_dot_data
from IPython.display import Image
from sklearn.tree import export_graphviz

In [ ]:
feature_names = X.columns.tolist()
target_name = np.array(['No', 'Yes'])

In [ ]:
dot_data = export_graphviz(tree,
                           filled=True,
                           rounded=True,
                           class_names=target_name,
                           feature_names=feature_names,
                           out_file=None)
graph = graph_from_dot_data(dot_data)
# 이미지로 생성
graph.write_png('tree.png')

In [ ]:
# 이미지 출력
data_graph = pydotplus.graph_from_dot_data(dot_data)
Image(data_graph.create_png())

In [ ]:
# 교차 검정
scores = cross_validate(estimator=tree,
                        X=X_train,
                        y=y_train,
                        scoring=['accuracy'],   # class가 2개 이상일 때는 accuracy만 사용 가능
                        cv=10,
                        n_jobs=-1,
                        return_train_score=False)

print(f'CV 정확도 점수 : {scores["test_accuracy"]}')    # validation을 통해 구한 값의 accuracy
print(f'CV 정확도: {np.mean(scores["test_accuracy"]):3} +/- {np.std(scores["test_accuracy"]):3}')

In [ ]:
# pipeline model
pipe_tree = make_pipeline(DecisionTreeClassifier())

In [ ]:
pipe_tree.get_params().keys()

In [ ]:
# 학습 곡선
param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

train_sizes, train_scores, test_scores = \
    learning_curve(estimator=pipe_tree,     # 수정
                   X=X_train,
                   y=y_train,
                   train_sizes=np.linspace(0.1, 1.0, 10),
                   cv=10,
                   n_jobs=1)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean,
         color='blue', marker='o',
         markersize=5, label='training accuracy')

plt.fill_between(train_sizes,
                 train_mean+train_std,
                 train_mean-train_std,
                 alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean,
         color='green', linestyle='--',
         marker='s', markersize=5,
         label='validation accuracy')

plt.fill_between(train_sizes,
                 test_mean+test_std,
                 test_mean-test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel("Number of Training Samples")
plt.ylabel("Accuracy")
plt.legend(loc='lower right')
plt.ylim([0.5, 1.03])   # 수정
plt.show()

In [ ]:
# 검증 곡선
param_range1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   # 수정
param_range2 = [10, 20, 30, 40, 50]   # 수정

train_scores, test_scores = \
    validation_curve(estimator=pipe_tree,     # 수정
                    X=X_train,
                    y=y_train,
                    param_name='decisiontreeclassifier__max_depth', # 수정
                    param_range=param_range,
                    cv=10,
                    n_jobs=1)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean,
         color='blue', marker='o',
         markersize=5, label='training accuracy')

plt.fill_between(param_range,
                 train_mean+train_std,
                 train_mean-train_std,
                 alpha=0.15, color='blue')

plt.plot(param_range, test_mean,
         color='green', linestyle='--',
         marker='s', markersize=5,
         label='validation accuracy')

plt.fill_between(param_range,
                 test_mean+test_std,
                 test_mean-test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel("Number of Max_depth")   # 수정
plt.legend(loc='lower right')
plt.xlabel("Parameter of Max_depth")    # 수정
plt.ylabel("Accuracy")
plt.ylim([0.8, 1.00])   # 수정
plt.tight_layout()
plt.show()

In [ ]:
# 하이퍼파라미터 튜닝
param_range1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # 수정
param_range2 = [10, 20, 30, 40, 50]    # 수정

param_grid = [{'decisiontreeclassifier__max_depth': param_range1,    # 수정
               'decisiontreeclassifier__min_samples_leaf': param_range2}]    # 수정

gs = GridSearchCV(estimator=pipe_tree,  # 수정
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)

In [ ]:
# 최적화 모델 검증
best_tree = gs.best_estimator_
best_tree.fit(X_train, y_train)

In [ ]:
y_pred = best_tree.predict(X_test)